# Fine Tuning A BERT Model With HuggingFace

## 1. Introduction

In [ ]:
import arxiv 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## 2. Collecting The Data 

In [ ]:


client = arxiv.Client()

# artificial intelligence
ai_results = [{
                "id": res.entry_id,
                "code": res.primary_category,
                "text": res.summary
              } for res in client.results(
                arxiv.Search(
                    query = "cat:cs.AI",
                    max_results = 1000
                    )
                )
]


In [ ]:
# robotics
ro_results = [{
                "id": res.entry_id,
                "code": res.primary_category,
                "text": res.summary
              } for res in client.results(
                arxiv.Search(
                    query = "cat:cs.RO",
                    max_results = 100
                    )
                )
]

In [ ]:
# information retervial
ir_results = [{
                "id": res.entry_id,
                "code": res.primary_category,
                "text": res.summary
              } for res in client.results(
                arxiv.Search(
                    query = "cat:cs.IR",
                    max_results = 1000
                    )
                )
]

In [ ]:

df = pd.DataFrame(
    ai_results + ir_results + ro_results
)

NameError: name 'df' is not defined

In [ ]:

labeler  = LabelEncoder()

In [120]:
df = df.assign(label=labeler.fit_transform(df["code"]))

In [123]:
labeler.classes_

array(['cs.AI', 'cs.IR', 'cs.RO'], dtype=object)

In [14]:
df.to_json("gs://harmon-arxiv/abstracts.json",
            storage_options={"token": "credentials.json"})

In [ ]:
# df = pd.read_json("gs://harmon-arxiv/abstracts.json",
#             storage_options={"token": "credentials.json"})

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], 
                                                    df["label"],
                                                     test_size=0.15, 
                                                     random_state=42,
                                                     stratify=df["label"]
                                                     )

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=y_train)

In [161]:
train_df = pd.DataFrame({"text": X_train,
                        "label": y_train})

val_df = pd.DataFrame({"text": X_val,
                        "label": y_val})

test_df = pd.DataFrame({"text": X_test,
                        "label": y_test})

In [162]:
train_df.shape

(1428, 2)

In [163]:
val_df.shape

(357, 2)

In [164]:
test_df.shape

(315, 2)

In [ ]:
test_df["text"].apply(lambda x: len(x)).to_frame("count").query("count == 0")

In [165]:
train_df.to_json("gs://harmon-arxiv/train_abstracts.json",
            storage_options={"token": "credentials.json"})

val_df.to_json("gs://harmon-arxiv/val_abstracts.json",
            storage_options={"token": "credentials.json"})

test_df.to_json("gs://harmon-arxiv/test_abstracts.json",
            storage_options={"token": "credentials.json"})

,count


## 2. HuggingFace Models

In [1]:
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
import evaluate 

/Users/mikeharmon/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_df = pd.read_json("gs://harmon-arxiv/train_abstracts.json",
                         storage_options={"token": "credentials.json"})

val_df = pd.read_json("gs://harmon-arxiv/val_abstracts.json",
                     storage_options={"token": "credentials.json"})

test_df = pd.read_json("gs://harmon-arxiv/test_abstracts.json",
                        storage_options={"token": "credentials.json"})

In [ ]:
# class_labels = ClassLabel(names=["artifical intelligence", "information retrival", ""])

In [5]:
train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
val_dataset = Dataset.from_pandas(val_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

In [6]:
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [16]:
test_dataset.features

{'text': Value('string'), 'label': Value('int64')}

In [7]:

checkpoint = "distilbert/distilbert-base-uncased"
device="mps"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
roc_auc_score = evaluate.load("roc_auc", "multiclass")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model = model.to(device)

In [9]:

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 315/315 [00:00<00:00, 9043.22 examples/s]


In [10]:
tokenized_datasets = tokenized_datasets.remove_columns("text")
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [13]:
tokenized_datasets = tokenized_datasets.with_format("torch")

In [14]:
val_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
model.eval()
for batch in val_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
        roc_auc_score.add_batch(references=batch["labels"], 
                               prediction_scores=scores)

In [28]:
roc_auc_score.compute(multi_class="ovr")

{'roc_auc': 0.1680612351892629}

## 3. Fine Tuning DistilBERT

In [29]:
model.device

device(type='mps', index=0)

In [30]:
# class_labels = ClassLabel(names=[
#                       "artifical intelligence", 
#                       "information retrival", 
#                       "robotics"])

In [17]:
def compute_metrics(eval_preds):
    roc_auc_score = evaluate.load("roc_auc", "multiclass")
    preds, labels = eval_preds
    scores = torch.nn.functional.softmax(
                        torch.tensor(preds), dim=-1)
    
    return roc_auc_score.compute(prediction_scores=scores, references=labels, multi_class="ovr")

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    logging_strategy="epoch"
)

In [20]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)


In [21]:
trainer.train()

/Users/mikeharmon/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 9.02 GB, other allocations: 9.08 GB, max allowed: 18.13 GB). Tried to allocate 89.42 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [26]:
trainer.save_model("temp")

In [ ]:
trainer.load   ("temp")

In [24]:
model = trainer.model
roc_auc_score = evaluate.load("roc_auc", "multiclass")


In [ ]:

for batch in val_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
        roc_auc_score.add_batch(references=batch["labels"], 
                               prediction_scores=scores)

RuntimeError: MPS backend out of memory (MPS allocated: 9.02 GB, other allocations: 9.14 GB, max allowed: 18.13 GB). Tried to allocate 8.48 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [22]:
predictions = torch.tensor(trainer.predict(tokenized_datasets["test"]))

scores = torch.nn.functional.softmax(
    torch.tensor(predictions.predictions), dim=-1)

RuntimeError: MPS backend out of memory (MPS allocated: 9.02 GB, other allocations: 9.14 GB, max allowed: 18.13 GB). Tried to allocate 8.09 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
roc_auc_score.compute(references=test_dataset["label"], 
                      prediction_scores=scores,
                      multi_class="ovr")

{'roc_auc': 0.8106397306397306}